#### Learning List
* [Effective Mordern C++](https://learning.oreilly.com/library/view/effective-modern-c-zhong-wen-ban/9787519817749/chapter-62.html)
* [Effective C++](https://gaozhiyuan.me/wp-content/uploads/2020/02/EFFECTIVE-C-%E6%94%B9%E5%96%84%E7%A8%8B%E5%BA%8F%E4%B8%8E%E8%AE%BE%E8%AE%A1%E7%9A%8455%E4%B8%AA%E5%85%B7%E4%BD%93%E5%81%9A%E6%B3%95-%E7%AC%AC3%E7%89%88-%E4%B8%AD%E6%96%87%E7%89%88.pdf)
* [Morden C++](https://changkun.de/modern-cpp/zh-cn/01-intro/)
* Primer C++ Plus


### 补充：字符串
C++ 处理字符串有两种方式
* C-style string
    * 字符串，即char 数组，并以空字符 (null character, \0，ASCII码为0) 结尾，用来标记字符串结尾

```c++
char cat[4] = {'c', 'a', 't', '\0'};

// 为了简便，使用 string literal 代替初始化列表，将字符数组初始化为字符串
char fish[] = "fish";
```

* string 类
    * c17 string_view


In [1]:
#include <iostream>
using namespace std;

// 字符数组所需的的长度 实际上是字符数 + 1, 即结尾的 \0
char fish[] = "fish";
cout << "Last value in c-styple string: " << int(fish[4]) << endl;

Last value in c-styple string: 0


### I. 迈向现代C++
#### 1.1 被弃用的特性
* string literal 字面常量不再允许被赋值给 char *, 而应该是 const char *

In [1]:
char *str = "Hello World";

input_line_7:2:14: warning: ISO C++11 does not allow conversion from string literal to 'char *' [-Wwritable-strings]
 char *str = "Hello World";
             ^


In [3]:
const char *str1 = "Hello World";
auto str2 = "Hello World";
std::string str3 = "Hello World";

auto str4 = u8"Hello World";  // UTF-8 string literal
auto str5 = u"Hello World";  // UTF-16 string literal
auto str6 = U"Hello World";  // UTF-32 string literal


#include <typeinfo>
#include <string>

std::cout << std::boolalpha << (typeid(str2) == typeid(const char*)) << std::endl;

std::cout << std::boolalpha << (typeid(str4) == typeid(const char*)) << std::endl;
std::cout << std::boolalpha << (typeid(str5) == typeid(const char16_t*)) << std::endl;
std::cout << std::boolalpha << (typeid(str6) == typeid(const char32_t*)) << std::endl;

//std::cout << decltype(str6) << std::endl;

true
true
true
true


* C++98 异常说明、 unexpected_handler、set_unexpected() 等相关特性被弃用，应该使用 **noexcept**。
* auto_ptr 被弃用，应使用 unique_ptr。
* register 关键字被弃用，可以使用但不再具备任何实际含义。C17中弃用

In [4]:
 // register 这个关键字的意思就是告诉编译器，最好把该关键字修饰的变量放在寄存器内, 弃用
register char i; 

input_line_13:3:1: error: ISO C++17 does not allow 'register' storage class specifier [-Wregister]
register char i; 
^~~~~~~~~


Interpreter Error: 

* bool 类型的 ++ 操作被弃用。
* 如果一个类有析构函数，为其生成拷贝构造函数和拷贝赋值运算符的特性被弃用了。??
* C 语言风格的类型转换被弃用（即在变量前使用 (convert_type)），应该使用 **static_cast**、**reinterpret_cast**、**const_cast**, **dynamic_cast** 来进行类型转换。
* 特别地，在最新的 C++17 标准中弃用了一些可以使用的 C 标准库，例如 \<ccomplex\>、\<cstdalign\>、\<cstdbool\> 与 \<ctgmath\> 等

### 补充：类型转换（type casting)
##### 隐式转换
编译器在没有任何显示操作符的干预下，对对象类型进行转换
* 数字类型之间：short到int，int到float，...
* 指针类型
    * 数组，函数隐式转换为指针
    * nullptr 转换为任意类型的指针
    * 任意类型的指针转换为void 指针
    * 指针向上提升：派生类指针转换为基类

##### 显示转换
* c-stype cast
* c++ 四种强制转换类型
    * static_cast
    * const_cast
    * dynamic_cast
    * reinterpret_cast

In [ ]:
uint ui = 100;
char a = (int)ui;

char b = static_cast<char>(ui);

### 补充：volatile
C/C++ 中的 volatile 关键字和 const 对应，用来修饰变量
```c++
volatile int value;
```

```
A volatile specifier is a hint to a compiler that an object may change its value in ways not specified by the language so that aggressive optimizations must be avoided.
```
当要求使用 volatile 声明的变量的值的时候，系统总是重新从它所在的内存读取数据，即使它前面的指令刚刚从该处读取过数据。而且读取的数据立刻被保存。比如，当变量声明为 volatile 系统不会从寄存器中读取它的值，而是直接去内存地址读取。

##### 用法
1) 中断服务程序中修改的供其它程序检测的变量需要加 volatile；
2) 多任务环境下各任务间共享的标志应该加 volatile；
3) 存储器映射的硬件寄存器通常也要加 volatile 说明，因为每次对它的读写都可能由不同意义；

##### volatile 指针
```c++
volatile char *vpch;
```

##### 多线程与 volatile
有些变量是用 volatile 关键字声明的。当两个线程都要用到某一个变量且该变量的值会被改变时，应该用 volatile 声明，该关键字的作用是防止优化编译器把变量从内存装入 CPU 寄存器中。如果变量被装入寄存器，那么两个线程有可能一个使用内存中的变量，一个使用寄存器中的变量，这会造成程序的错误执行。

### 1.2 与C的兼容性

decltype

In [2]:
float balance[10]{} // initialized list, all element set to 0

# Effective Morden C++
## Chapter 1. 类型推导
### Term 3. decltype


In [4]:
#include <string>
auto t = [](const string& s) { cout << s << endl;  };

decltype(t) t2 = t;   // decltype(t)是 void (const string&)

t2("hello")

hello


## Chapter 3. Morden C++
### Term 4.  在创建对象时注意区分()和{}

In [2]:
int x{5};
// int x{5.0};  //type 'double' cannot be narrowed to 'int' in initializer list [-Wc++11-narrowing]

In [3]:
class Position {
public:    
    Position(int x, int y): x(x), y(y){}

private:
    int x;
    int y;
};

class Widget {
    enum class Type {
        Rectangle, Circle
    };
    
    Type type;
    Position p{0, 0};
};

## 黑马 C++ 提高

#### Typedef 
别名， 作用：

1）简便
```c++
typedef shared_ptr<const char *> pStr;
```

2）有利于程序的一致性

In [8]:
typedef int DataType;
DataType number = 10;
cout << typeid(number).name() << endl;

i
